# 1. Notebook Setup
# Title, assignment info, and markdown overview.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import csv
import math
from typing import Union

In [ ]:
# 2. Imports


class Task:
    time:int =0
    difficulty:int =0
    deadline:int = 0
    required_skill:str = ''
    def __str__(self):
        return f"T Time:{self.time} Difficulty:{self.difficulty} Deadline:{self.deadline} requiredSkill:{self.required_skill}"

class Employee:
    available_hours:int = 0
    skill_level:int = 4
    skills:list[str]=['']

    def __str__(self):
        return f"E AH:{self.available_hours} Skill-Level:{self.skill_level} Skills:{self.skills}"

TDMapping = list[Union[str|int]]

# ask ID Time (hrs) Difficulty Deadline (hrs) Required Skill
# T1 4 3 8 A
# T2 6 5 12 B
# T3 2 2 6 A
# T4 5 4 10 C
# T5 3 1 7 A# ask ID Time (hrs) Difficulty Deadline (hrs) Required Skill
# T1 4 3 8 A
# T2 6 5 12 B
# T3 2 2 6 A
# T4 5 4 10 C
# T5 3 1 7 A
# T6 8 6 15 B
# T7 4 3 9 C
# T8 7 5 14 B
# T9 2 2 5 A
# T10 6 4 11 C
# T6 8 6 15 B
# T7 4 3 9 C
# T8 7 5 14 B
# T9 2 2 5 A
# T10 6 4 11 C


In [ ]:
# I/O
"""
Task csv:

'ID', 'Time (hrs)', 'Difficulty', 'Deadline (hrs)', 'Required Skill'

Employee csv:
'Employee ID', 'Available Hrs', 'Skill Level', 'Skills'


Using an adjancey list instead of an adjacney matrix!

Since the input vector is 10 mappings x 11 features. Including a unique assignment penalty is redundant since a task mapped to 2 employees will create (10+1) mappings which doesn't work for the network input layer!
"""


mapping_file_path = "data/Mappings.csv"
employees_file_path = "data/Employee.csv"
tasks_file_path = "data/Tasks.csv"

class DataLoader:
    num_of_tasks =10
    num_of_employees = 5
    tasks:list[Task]=[]
    employees:list[Employee]=[]
    
    def load_tasks(self,fileName=tasks_file_path):
        tasks =[]
        with open(fileName,'r') as csvfile:
            taskReader = csv.reader(csvfile)
            next(taskReader)
            for taskArr in taskReader:
                newT = Task()
                newT.time=int(taskArr[1])
                newT.difficulty= int(taskArr[2])
                newT.deadline = int(taskArr[3])
                newT.required_skill= taskArr[4]
                tasks.append(newT)
        self.tasks= tasks

    def load_employees(self,fileName=employees_file_path):
        employees=[]
        with open(fileName,'r') as csvfile:
            employeeReader = csv.reader(csvfile)
            next(employeeReader)
            for employeeArr in employeeReader:
                newEmployee = Employee()
                newEmployee.available_hours=int(employeeArr[1])
                newEmployee.skill_level= int(employeeArr[2])
                skills = employeeArr[3].split(',')
                newEmployee.skills = skills
                employees.append(newEmployee)
        self.employees = employees
    def loadAll(self):
        self.load_employees()
        self.load_tasks()
    


class MappingHandler:
    

    def __init__(self,tasks:list[Task],employees:list[Employee],num_of_mappings=100):
        self.tasks:list[Task]= tasks
        self.employees:list[Employee] = employees
        self.num_of_mappings = num_of_mappings
        self.mappings = []
        self.costs=[]


    def __costFunction(self,Mapping:list[int]):
        w=0.2
        overload = 0
        skill_mismatch = 0
        difficulty_violation = 0
        deadline_violation = 0
        unique_assignment = 0
        employee_task_adj_list:list[list[Task]]=[[] for _ in range(len(self.employees))]

        for i in range(len(Mapping)):
            taskId=i

            task = self.tasks[taskId]
            num_of_employees_assigned = 0
            employeeId = Mapping[i]
            num_of_employees_assigned+=1
            employee = self.employees[employeeId]
            employee_task_adj_list[employeeId].append(task)
            # skill mismatch violation
            if task.required_skill not in employee.skills:
                skill_mismatch+=1
            if task.difficulty> employee.skill_level:
                difficulty_violation+= 1
            unique_assignment+= max(0,num_of_employees_assigned-1)



        for employeeId in range(len(employee_task_adj_list)):
            employee = self.employees[employeeId]
            sortedEmployeeTasks = employee_task_adj_list[employeeId]
            sortedEmployeeTasks.sort(key= lambda x:x.time)
            sumHours=0
            finishTime=0
            for t in sortedEmployeeTasks:
                sumHours+=t.time
                finishTime+= t.time
                deadline_violation+= max(0,finishTime - t.deadline)
            overload+= max(0,sumHours- employee.available_hours) 
        total_penalty = overload+skill_mismatch+unique_assignment+deadline_violation+ difficulty_violation
        print(overload)
        return round(total_penalty*w,3)
    def __reset(self):
        self.mappings=[]
        self.costs=[]


    def generateMappings(self):
        unique = set()
        self.__reset()
        while len(unique) < self.num_of_mappings:
            mapping:list[int] = [0 for _ in self.tasks]         # Create a mapping for each iteration. Using an adjlist format
            # possible_assignments = [i for i in range(len(employees))]  #list of possible assignments
            for taskId in range(len(self.tasks)):
                rand_employee =rd.randint(0,len(self.employees)-1)
                mapping[taskId]=rand_employee
            string = str(mapping)
            if string not in unique: # ensure no duplicates in data generation
                unique.add(string)
                self.mappings.append(mapping)
                self.costs.append(self.__costFunction(mapping))
    

    def readCSV(self,fileName=mapping_file_path):
        self.__reset()
        with open(fileName,'r') as csvfile:
            mappingReader = csv.reader(csvfile)
            next(mappingReader)
            for line in mappingReader:
                newMapping:list[int] = [int(x) for x in line]
                self.mappings.append(newMapping)
                self.costs.append(self.__costFunction(newMapping)) 


    def writeCSV(self,mappings:list[list[int]],filepath =mapping_file_path):
        with open(filepath,'w',newline='') as csvfile:
                mappingWriter = csv.writer(csvfile)
                mappingWriter.writerow(["T1",'T2','T3','T4','T5','T6','T7','T8','T9','T10','Penalty'])
                mappingWriter.writerows(mappings)
        





def testDataGen():
    dl = DataLoader()
    dl.loadAll()
    mappingloader = MappingHandler(dl.tasks,dl.employees)
    print(dl.tasks)
    mappingloader.generateMappings()

testDataGen()


# def TestDataGen():

#     all_mappings = mappingGenerator(task,employees)
#     for mapping in all_mappings:
#         cost = costFunction(mapping,task,employees)
#         print(cost)

# def verifyCost():
#     mapping=[[2],[3],[1],[4],[2],[5],[1],[3],[5],[4]]
    
#     for i in range(len(mapping)):
#         mapping[i][0]-=1
#     employees,tasks = DataLoader().loadAll()
#     # for t in tasks:
#     #     print(t)
    
#     for e in employees:
#         print(e)
#     print(costFunction(mapping,tasks,employees))





[<__main__.Task object at 0x0000020FC6A2DFD0>, <__main__.Task object at 0x0000020FC6A2EB70>, <__main__.Task object at 0x0000020FC6A2ED20>, <__main__.Task object at 0x0000020FC6A2E7E0>, <__main__.Task object at 0x0000020FC6A2DA00>, <__main__.Task object at 0x0000020FC6A2E120>, <__main__.Task object at 0x0000020FC6A2D2B0>, <__main__.Task object at 0x0000020FC6A2F740>, <__main__.Task object at 0x0000020FC6A2C9B0>, <__main__.Task object at 0x0000020FC6A2E630>]
17
9
9
14
3
10
15
17
10
4
18
10
19
4
12
16
17
2
2
12
14
8
13
10
1
10
9
5
18
19
0
15
13
15
10
16
10
11
17
7
8
15
12
8
19
6
7
16
14
3
13
14
11
15
10
15
8
9
16
4
11
15
15
9
3
9
11
14
13
11
11
18
5
15
14
10
11
21
12
13
5
14
20
11
15
19
18
8
18
5
8
13
6
7
5
9
9
4
9
2


In [ ]:
#data processing and I/O




def dataLoad():



# 3. Data Generation/Load
#- Generate or load the 100 mappings CSV
#- Load into pandas DataFrame
# 4. Preprocessing
def one_hot_encode_skill(skills):
    pass
# returns a 3-element vector
...
def construct_input_vector(mapping_row):
    pass
# builds 110-dim vector for one example

# 5. Model Definitions
class NeuralNetwork:
    def __init__(self, layer_dims, activation=’relu’):
        pass

    def forward(self, x):
        pass

    def backward(self, x, y_true):
        pass

    def update_params(self, lr):
        pass

    # 6. Training Loop
    def train(model, X_train, y_train, params):
        pass
    # implement mini-batch SGD, record loss

# 7. Evaluation & Plots
#- Generate the eight required figures
#- Save each via plt.savefig()
# 8. Save & Export
#- Download figures
#- Optionally, pickle model parameters